# Lab 3: Language modeling

In this week's lab, we'll explore techniques for language modeling. 

The aims of this lab are:
*   Model the probability of generating language with unigrams and trigram LMs
*   Evaluate the quality of language model using perplexity
*   Understand and address issues of sparsity in language modeling
*   Experiment with applications of language models in understanding text

## Background: Load Reddit
Just run the cells below, it should be familiar by now. 

In [1]:
local_file = "coarse_discourse_dump_reddit_unfiltered.json"
#!gsutil cp gs://textasdata/coarse_discourse_dump_reddit.json $local_file
#!curl -o $local_file https://storage.googleapis.com/tad2018/coarse_discourse_dump_reddit.json

In [3]:
import warnings
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import json
import pandas as pd

posts = list()

# If the dataset is too large, you can load a subset of the posts.
post_limit = 100000000

# Construct a dataframe, by opening the JSON file line-by-line
with open(local_file) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    if (len(posts) > post_limit):
      break
      
    for post in thread['posts']:
      posts.append((thread['subreddit'], thread['title'], thread['url'],
                        post['id'], post.get('author', ""), post.get('body', "")))
print(len(posts))

labels = ['subreddit', 'title', 'id', 'url', 'author', 'body']
post_frame = pd.DataFrame(posts, columns=labels)


ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 192 from PyObject

In [0]:
#!python -m spacy download en

import spacy

# Load the small english model. 
# Disable the advanced NLP features in the pipeline for efficiency.
nlp = spacy.load('en_core_web_sm', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

#@Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#@Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    if (not token.is_punct):
#    if (token.is_alpha or token.is_digit):
      normalized = token.text.lower().strip()
      normalized_tokens.append(normalized)
  return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))  

It may take several minutes to run spaCy on all posts in the collection.

In [0]:
# Use the tokenizer to extract all tokens from the body of the posts.
# Flatten the tokens in the post into a single list of all the tokens.
import itertools
all_tokens = []
all_posts_tokenized = post_frame.body.apply(tokenize_normalize)
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))
print("Num tokens: ", len(all_tokens))

## Unigram language model

We'll use the dictionary from Lab 2 as a building block to build a unigram language model.  We added special padding tokens (for start /end sentence) that will be used later. 

In [0]:
import collections

class SimpleDictionary(object):

  # Add special "padding" tokens as well as unk.
  START_TOKEN = "<p>"
  END_TOKEN = "</p>"
  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    self.token_counts = collections.Counter(tokens)
    self.N = sum(iter(self.token_counts.values()))

    # Leave space for "<p>", "</p>", and "<unk>"
    top_counts = self.token_counts.most_common(None if size is None else (size - 3))
    vocab = ([self.START_TOKEN, self.END_TOKEN, self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in iter(self.id_to_word.items())}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience create the vocab set
    self.vocab_set = set(iter(self.word_to_id.keys()))

    # Store special IDs
    self.START_ID = self.word_to_id[self.START_TOKEN]
    self.END_ID = self.word_to_id[self.END_TOKEN]
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]
  
  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def sentence_to_ids(self, words):
    return [self.START_ID] + self.words_to_ids(words) + [self.END_ID]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))

In [0]:
dictionary = SimpleDictionary(all_tokens)
print("Vocabulary size: %d unique words" % dictionary.size)

#### Your task

- Create a class, ``UnigramLM``
- Initializer should take a dictionary (``SimpleDictionary``)
- It should pre-compute the unigram probabilities for all the words in the `vocab` and store them in a dictionary
- Keep a copy of the `vocab_set` from the dictionary as a member variable for convenience.
- Unknown (OOV) words should return a probability of 0.  Consider using the `defaultdict` collection with a 0.0 lambda.
- **Recall from lecture **: What are the two key functions that a language model must satisfy? Define a function for each of these.  
  
Click SHOW CODE below for a skeleton (and required function names).  


In [0]:
#@title
from collections import defaultdict
import numpy as np

class UnigramLM(object):
  
  def __init__(self, dictionary):
    # Compute the probabilities and store them in a data structure.
    self.vocab_set =
    
  # Compute the conditional probability of the next token for the unigram model. 
  def next_token_conditional_prob(self, previous_words, next_word):
    
  # Compute the probability of a sequence, P(w1, ..., wn)
  # When Verbose is true, it prints the probability for each
  # token in the sequence.
  def sequence_probability(self, sequence, verbose=False):

In [0]:
unigram_lm = UnigramLM(dictionary)

Let's try computing the unigram probabilities of words. 

In [0]:
# Try computing the unigram probabilities for indivudal words.
print("Pr(the):", unigram_lm.next_token_conditional_prob(None, 'the'))
print("Pr(glasgow):", unigram_lm.next_token_conditional_prob(None,'glasgow'))
print("Pr(defenestrate):", unigram_lm.next_token_conditional_prob(None,'defenestrate'))

# And for the same word, but with different contexts
print(unigram_lm.next_token_conditional_prob(["the", "end", "of", "the", "world"], "as"))
print(unigram_lm.next_token_conditional_prob(["the"], "as"))



The probabilities should be about 3.5% for 'the' and 0 for defenstrate. 

Now, try computing the probability of a sequence of words.  

In [0]:
probability = unigram_lm.sequence_probability(["the", "end", "of", "the", "world", "as", "we", "know", "it"], True)
print('{:.40f}'.format(probability))

It should be about 1.8×10-21. This is already a very small number and most of the values in these sequence are large by typical probability values. As a result, we usually do the probability computation in log space in practice to avoid problems of underflow.

### Application: Spelling correction

We can use this simple language model to build a spelling corrector.

Below is a spelling corrector based on code by Peter Norvig, a director of research at Google.

#### Your task
- Fill in the `P` function below to compute the unigram probability of a word.  Hint: use probability from the language model.
- Modify `candidates` to generate the set of candidate words 
 - The words can be up to two edits away. See the provided edit distance functions in the class. 
 - Word candidates are `known` (in the vocab).

In [0]:
class SimpleSpellingCorrector(object): 
  
  def __init__(self, lm):
    self.lm = lm
  
  def P(self, word): 
    "Unigram probability of `word`."
    return # YOUR CODE HERE

  def correction(self, word): 
    "Most probable spelling correction for word."
    return max(self.candidates(word), key=self.P)

  def candidates(self, word): 
    "Generate possible spelling corrections for word"
    return # YOUR CODE TO GENERATE CANDIDATES

  def known(self, words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in self.lm.vocab_set)

  def edits1(self, word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

  def edits2(self, word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

Now, let's test how the full spelling corrector works. 

In [0]:
word = "speling"
#word = TRY YOUR WORD

corrector = SimpleSpellingCorrector(unigram_lm)
candidates = corrector.candidates(word)
print("Spelling candidates for:", word)
for candidate in candidates:
  print("{} \t{:.9f}".format(candidate, corrector.P(candidate)))

# The correct takes the word that has the highest probability (occurs most often).
correction = corrector.correction(word)
print("\nSelected correction:", correction, corrector.P(correction))

Wohooo! We have a very basic reddit spelling corrector. 
Try it yourself on other words. 

However, as we'll see it's not perfect.  Let's look at a sequence of words below.

In [0]:
# Simple function that takes a string as input
# and runs the spelling corrector for each token. 
# Returns a list of spell-corrected tokens.
def spell_correct(string, corrector):
  corrections = list()
  tokens = tokenize_normalize(string)
  for t in tokens: 
    correction = corrector.correction(t)
    corrections.append(correction)
  return corrections 

string = "it is amazon"
print(spell_correct(string, corrector))

string = "http www amazin"
print(spell_correct(string, corrector))

Not so 'amazing'. In both of these cases, we estimated the likelihood of spelling correcting each word on its own,  independently.  This fails in obvious cases where the correct word should be apparent given the sequence. 

We can do better by taking two factors into account:

1.   Computing the probability of the whole sequence of words (in case there are multiple spelling mistakes) 
2.   Using word context (bi-grams and trigrams) to improve the word probablility estimate

We'll look at how to compute both in the next section. 

# N-Gram Language Models

The unigram model isn't a very good one - it doesn't model any previous context. On the other hand, we can't model _all_ of the preceding words, because that history will get prohibitively long and extremely sparse. As a compromise, we make a _Markov assumption_ and limit ourselves to a finite history of $n$ words.


#### Thought exercise
- For a corpus of 1 million tokens and a vocabulary size of 10,000:
 - What is the maximum number of bigrams that could theoretically exist? 
 - How many possible trigrams?

Think about this process as drawing a word, then another from the collection in order. 

In [0]:
N = 1000000
V = 10000

# Number of possible bigrams
bigrams = 
print(bigrams)

# Number of possible trigrams
trigrams = 
print(trigrams)

# Technically, the practical limit is also limited by size of the collection, N.
# Bigrams would be limited by N-1
# Trigrams would be limited by N-2
print(min(N-1, bigrams))
print(min(N-2, trigrams))

#### Your task 
- Print number of possible bigrams and trigrams based on the size of the Reddit `dictionary`. 


- How much space would a table of all possible unigrams, bigrams and trigrams take? 
- For simplicity, let's assume 8 bytes per entry.  
- How much space would we need to store all of them? 

In [0]:
import psutil
print ("Vocab size: %d words" % (dictionary.size))
print ("Unigrams need: %g KB" % (8 * (dictionary.size ** 1) / (2**10)))
print ("Bigrams need:  %g GB" % (8 * (dictionary.size ** 2) / (2**30)))
print ("Trigrams need:  %g TB" % (8 * (dictionary.size ** 3) / (2**40)))
print ("Available:     %g GB" % (psutil.virtual_memory().available / (2**30)))

Look at the results! Given a typical machine, we can store all possible unigrams and some of the bigrams, but we'd never get to trigrams!

Thankfully, we don't have to store every possible n-gram combination. Most words occur rarely. As a result, for bigrams and trigrams, the table will be very sparse. We will only store the entries that we observe; the rest we can taken to be zero or we can estimate their values using smoothing.



### Constructing our Model
For now, we'll build a trigram model, which considers the two preceding words:

$$ P(w_i\ |\ w_{i-1}, ..., w_0) \approx P(w_i\ |\ w_{i-1}, w_{i-2}) $$

We'll need to store a table of the word probabilities, indexed by triples $(w_i, w_{i-1}, w_{i-2})$. 


We'll represent our model with a nested map `context => word => probability`, where word is $w_i$ and for our trigram model, the context is the two preceding words $(w_{i-1}, w_{i-2})$ .

First, we'll go through the corpus and compute raw trigram counts $c(abc)$, which we'll then normalize into Maximum Likelihood Estimates of the probabilities:

$$  P_{abc} = P(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{\mathrm{c(abc)}}{\sum_{c'}\mathrm{c(abc')}} = \frac{C_{abc}}{\sum_{c'} C_{abc'}} $$

*Notational note:* sometimes we write $(w_{i-1}, w_{i-2})$  and sometimes $(w_{i-2}, w_{i-1})$, because P(c|ab) is more readable and natural than P(c|ba).  It doesn't matter as long as the counts are consistent.



#### Optional Task 
Note: This task may take awhile to complete.  It is **optional**, but goes into depth on how to implement an n-gram LM. Don't spend too long on it, be sure to leave enough time complete the rest of the lab. 
- Create a class, ``TrigramLM``
- Initializer should take a sequence of tokens and construct the model with n-gram counts
- The class should keep the ``vocab`` and its size, like ``SimpleDictionary``
- For convenience, keep counts of both the trigram ``counts`` (numerator) and the ``context_totals`` (denominator) values. 
- The pobability of a sequence should be computed using log_2 space 
- **Tip:** You might use Python's defaultdict collection (with a lambda) to set default values, particularly for multi-level maps
- **Tip: **Use tuples of tokens as keys in the dictionary, *not* lists of tokens

Click SHOW CODE to see the implementation.

In [0]:
#@title
from collections import defaultdict
import numpy as np

class SimpleTrigramLM(object):
   """Simple Trigram LM"""

  def __init__(self, tokens):
      """Build our trigram model.
      Args:
        tokens: (list or np.array) of training tokens
      Returns:
        None
      """    
    # Compute the counts and store them in a data structure.
    
  # Compute the conditional probability of the next token,  P(wi | wi-1, wi-2)
  def next_token_conditional_prob(self, previous_words, next_word):
      """Next token conditional probability.
      Args:
        word: (string) w in P(w | w_1 w_2 )
        seq: (list of string) [w_1, w_2, w_3, ...]
      Returns:
        (float) P_k(w | w_1 w_2), according to the model
      """    
    
  # Compute the probability of a sequence, P(w1, ..., wn)
  def sequence_probability(self, sequence):
      """Compute log probability (base 2) of the given sequence."""


In [0]:
#@title
from collections import defaultdict
import numpy as np

class SimpleTrigramLM(object):
    """Simple Trigram LM"""
    order_n = 3

    def __init__(self, tokens):
        """Build our trigram model.
        Args:
          tokens: (list or np.array) of training tokens
        Returns:
          None
        """
        print("Num tokens: ", len(tokens))
        
        # Raw trigram counts over the corpus.
        # c(w | w_1 w_2) = self.counts[(w_2,w_1)][w]
        # Be sure to use tuples (w_2,w_1) as keys, *not* lists [w_2,w_1]
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))

        # Map of (w_1, w_2) -> int
        # Entries are c( w_2, w_1 ) = sum_w c(w_2, w_1, w)
        self.context_totals = dict()

        # Track unique words seen, for normalization
        self.vocab = set()

        # Iterate through the word stream once
        # Compute trigram counts 
        # This is a sliding window over each word.
        w_1, w_2 = None, None
        for word in tokens:
            self.vocab.add(word)
            if w_1 is not None and w_2 is not None:
                self.counts[(w_2,w_1)][word] += 1
            # Update context
            w_2 = w_1
            w_1 = word

        for context, ctr in iter(self.counts.items()):  
            self.context_totals[context] = sum(iter(ctr.values())) 

        # Total vocabulary size, for normalization
        self.V = len(self.vocab)

    def next_token_conditional_prob(self, word, seq):
        """Next token conditional probability.
        Args:
          word: (string) w in P(w | w_1 w_2 )
          seq: (list of string) [w_1, w_2, w_3, ...]
        Returns:
          (float) P_k(w | w_1 w_2), according to the model
        """
        context = tuple(seq[-2:])  # (w_2, w_1)
        cw = self.counts.get(context, {}).get(word, 0)  
        numerator = cw 
        cc = self.context_totals.get(context, 0)  
        denominator = cc  
        return numerator / denominator  
      
    def sequence_probability(self, seq, verbose=False):
      """Compute log probability (base 2) of the given sequence."""
      context_size = self.order_n - 1
      score = 0.0
      count = 0
      # Start at third word, since we need a full context.
      for i in range(context_size, len(seq)):
        context = seq[i-context_size:i]
        context_prob = self.next_token_conditional_prob(seq[i], context)
        s = np.log2(context_prob)
        # DEBUG.
        if verbose:
            print ("log P(%s | %s) = %.03f" % (seq[i], " ".join(context), s))
        score += s
        count += 1
       
      return score, count      

Build the trigram model on all tokens, it should take 15-30s.

In [0]:
import time
t0 = time.time()
print ("Building trigram LM..."),
lm = SimpleTrigramLM(all_tokens)
print ("done in %.02f s" % (time.time() - t0))

Let's inspect the trigram output

In [0]:
print ("Most frequent tri-grams:")
flat_counts = defaultdict(lambda: 0.0)
# This code puts the contexts back together with their word counts
for (context, counts) in lm.counts.items():
  for (word, count) in counts.items():
    flat_counts[context, word] = count

sorted_trigrams = sorted(flat_counts.items(), key=lambda k_v: k_v[1], reverse=True)

for (word, count) in sorted_trigrams[:50]:
    print("\"%s\": %d" % (word, count))

The should be obvious, like "a lot of" -- sequence of frequent words.  However, given the large size of our corpus the counts are still quite small.

Let's look at how much memory is used in practice.


In [0]:
def print_stats(lm):
    """Output summary statistics about our language model."""
    print ("=== N-gram Language Model stats ===")
    unique_ngrams = sum(len(c) for k,c in lm.counts.items())
    print ("%g unique 3-grams" % (unique_ngrams))

    optimal_memory_bytes = sum(
            (4 * len(k) + 20 * len(v))
             for k, v in lm.counts.items())
    print ("Optimal memory usage (counts only): %d MB" %
            (optimal_memory_bytes / (2**20)))

print_stats(lm)

Despite the theoretical size, the number of unique bigrams and trigrams seen in practice is very sparse.

## Generating made up Reddit posts

- Language models are *generative*.  They model the probability of generating a sequence of words. The probabilities can be used in interesting ways, for example to score sequences or even to generate made up text sequences iteratively.  
- Below is a function, `sample_next` that samples possible next words randomly proportional to their conditional probability as the next word in the sequence. 



In [0]:
def sample_next(lm, seq):
    """Sample a word from the conditional distribution."""
    # This looks through each possible next word in the vocab and computes its 
    # conditional probability with the current sequence.
    probs = [lm.next_token_conditional_prob(token, seq) for token in lm.vocab]
    
    # Pick a word at random according to its conditional probability
    return np.random.choice(list(lm.vocab), p=probs)

We can use a Language Model to generate new made up data from our model. We'll generate words sequentially, one token at a time by trying to predict the next word in the sequence.

In [0]:
# Given it the start sequence to indicate the start of a post. 
   
def hallucinate_text(start, max_length):
  sequence = list()
  sequence.extend(start)
  for i in range(max_length - len(start)):
      sequence.append(sample_next(lm, sequence))
  #print (" ".join(sequence))
  #print ("[{1:d} tokens; log P(seq): {0:.02f}]".format(lm.sequence_probability(seq)))
  return sequence
    
# Maximum length of sequence to generate. 
max_length = 20

# Number of sequences to generate
num_sequences = 5


start = ["i", "feel"] # Needs to be an n-gram that occurs in our collection
for _ in range(num_sequences):
  sequence = hallucinate_text(start, max_length)
  print (" ".join(sequence))
    

We now have a simple bot that can write reddit-like posts!  It sort-of-almost makes sense. Many naive spam bots work in a similar way by taking sample text and using it to generate made up auto-replies. 

Have fun with this example by modifying the starting sequence.  Note: currently the starting sequence must occur in our input! You can also change the length of posts or the number of posts.  
 - Consider: If you were building an auto-reply system, how would you alter this to take the similarity to an existing starting post? Recall the lab from last lecture.

Let's evaluate our models by seeing how well they can predict real text.

## Language Model Evaluation

### Train and test sets

We'll split the data into two parts: train and test. We train our model (compute counts) on the training sample of posts. This means that we fixed (or *fit* in sci-kit learn terminology) our vocabulary based on the words in *training* set.  We then evaluate on how well it can predict the new unseen test posts.  


In [0]:
# We shuffle the posts randomly by calling Pandas sample function with a fraction of 1.
shuffled_posts = post_frame.sample(frac=1)

# Split the data into 80% train, 20% test posts.
train_frac = 0.8
split_idx = int(train_frac * len(shuffled_posts))
train_posts = shuffled_posts[:split_idx]
test_posts = shuffled_posts[split_idx:]

print ("Training set:", len(train_posts))
print ("Test set: ", len(test_posts))

Apply the tokenizer and dictionary to ONLY the training posts.

In [0]:
train_post_tokens = train_posts.body.apply(tokenize_normalize)
flat_train_tokens = list(itertools.chain.from_iterable(train_post_tokens))
train_flat_dictionary = SimpleDictionary(flat_train_tokens)


We're about ready to evaluate. But we do some extra text processing steps first. 

We need to deal with two issues:


1.   Add special delimiters to mark the start / end of posts. We want to model the probability of words starting or ending a post. We also want the probability of generating words in a post to be separate (independent) from each other. Note that the number of padding tokens (two at the start) depends on the order of the model (we are using a trigram).
2.   Handle UNK (OOV) tokens in new data and replace the words with UNK tokens. 

The result is a new `post_to_tokens` function.  Although we don't do it here for readability, this function could also convert the token sequence into a sequence of integer word IDs using the dictionary (like we did for a one-hot encoding).

In [0]:
# Replace unkown words with the special UNK word.
def replace_unk(word, wordset=None):
    if word in wordset: 
      return word 
    else: 
      return SimpleDictionary.UNK_TOKEN # unknown token
    
def posts_to_tokens(posts):
    """Returns an flattened list of the words in the posts, with padding for a trigram model."""
    # Pad each post with delimters.
    # Why do we use two start delimiters?
    padded_posts = ([SimpleDictionary.START_TOKEN, SimpleDictionary.START_TOKEN] + p + [SimpleDictionary.END_TOKEN] for p in posts)
    
    # This will replace anything not in vocab with <unk> 
    return np.array([replace_unk(w, wordset=train_flat_dictionary.vocab_set) 
                     for w in list(itertools.chain.from_iterable((padded_posts)))], dtype=object)

In [0]:
flat_delimited_train_tokens = posts_to_tokens(train_post_tokens)
train_dictionary = SimpleDictionary(flat_delimited_train_tokens)
train_lm = SimpleTrigramLM(flat_delimited_train_tokens)


## Perplexity

We'll score our model using perplexity. As mentioned in lecture, we take $ 2^H $ (exponentiate it) to get the perplexity score, where $H$ is the cross-entropy. How do we get this? 

Running the `sequence_probability` function computes the log-likelihood of our data: 

$$ Log P(w_1, ... w_N) = \sum_{i=1}^N \log_2 \hat{p}(w_i\ |\ w_{i-1}, w_{i-2}) $$

Which is very close to the cross-entropy loss:
$$ \text{H}_{\text{total}}(y, \hat{y}) = -1 \sum_{i=1}^N \frac{1}{N} \log_2 \hat{p}(w_i\ |\ w_{i-1}, w_{i-2}) $$

The cross-entropy is equal to $-1$ times the log-likelihood of the data under our model, averaged by the total number of instances (tokens).

Let's run it.  We compute the probability of the whole collection as a sequence using our model.

In [0]:
log_p_data, num_real_tokens = train_lm.sequence_probability(flat_delimited_train_tokens)
print ("Train perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens)))

#### Optional Task: 
- Compute the unigram perplexity of the training data (in log space)
- Use the UnigramLM with the train_dictionary
- Compare the unigram model perplexity vs the trigram model

*Hint:* Compute the probabilities of each word separately since context isn't needed for the unigram model.  Or modify your UnigramLM model to compute sequence probabilities in log space, as we did for the trigram model.


In [0]:
#@title
log_p = 0 
unigram_train_lm = UnigramLM(train_dictionary)
# We do this here, we could also modify our LM to operate in log space
# directly, which is what is commonly done.
for w in flat_delimited_train_tokens:
  prob = unigram_train_lm.next_token_conditional_prob(None, w)
  if prob == 0: 
    print (w)
  log_prob = np.log2(prob)
  log_p += log_prob
  
print ("Unigram train perplexity: %.02f" % (2**(-1*log_p/train_dictionary.N)))

You should find that the the unigram perplexity is slightly less than 1000. This means that on average at each word the model picks between on average 1000 words.  In contrast, you should find that the perplexity of the trigram model is around 10.  It reduces the uncertainty in picking the next word by a factor of 100! 

But, this is just on the training data.  How well do the models do on our test data:

In [0]:
test_post_tokens = test_posts.body.apply(tokenize_normalize)
flat_delimited_test_tokens = posts_to_tokens(test_post_tokens)

log_p_data, num_real_tokens = train_lm.sequence_probability(flat_delimited_test_tokens)
print ("Test perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens)))

Whoops!! What's going on here? Our model gets an absurdly low perplexity on the training data, but a divide by zero error (it means a sequence has a zero probability of occurring) on the test data. Why?

**Answer:** the n-gram model overfits without any smoothing; it thinks the generated sequences are completely likely and unseen sequences don't exist. 

## Smoothing and handling the unknown

Our simple model doesn't have any real mechanism for handling unknown words - if we feed something unseen it will result in an error:

Is assuming zero probabilities realistic? Let's look back at our unigram distribution:
- What percentage of the words in the test tokens are UNK?

In [0]:
print ("%% <unk> in test set: %.02f%%" % (np.sum(np.array(flat_delimited_test_tokens) == SimpleDictionary.UNK_TOKEN) * 100.0 / len(flat_delimited_test_tokens)))

In this case, we find about 1-2% of all tokens are unseen (depending on tokenization). If we want to use our language model in the wild, we'll need to apply smoothing.

### Laplace Add-k Smoothing smoothing

Remember that if a trigram hasn't been seen before, it will be assigned a zero probability. In this section we'll experiment with Laplace (Add-K) smoothing to fix that problem. Recall our unsmoothed maximum likelihood estimate of $ P(w_i\ |\ w_{i-1}, w_{i-2})$ where we use the raw distribution over words seen in a context in the training data:

$$  \hat{P}(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc}}{\sum_{c'} C_{abc'}} $$

Recall from lecture that Add-k smoothing is where we add $k > 0$ to each count $C_{abc}$, pretending we've seen every vocabulary word $k$ extra times in each context. So we have:

$$ \hat{P}_k(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc} + k}{\sum_{c'} (C_{abc'} + k)} = \frac{C_{abc} + k}{C_{ab} + k\cdot|V|} $$

where $|V|$ is the size of our vocabulary.

#### Optional exercise
NOTE: This exercise is OPTIONAL.  If you get stuck, run the working code to be able to try different values of K. 

- Copy your `SimpleTrigramLm` class and call it `AddKTrigramLM`
- Add support for a `k` smoothing member variable with a default value of 0.0
- Add a function to `set_k` so that it can be updated
- Modify `next_token_conditional_prob` to smooth the probability values with Laplace smoothing.


In [0]:
#@title
from collections import defaultdict
import numpy as np

class AddKTrigramLM(object):
    """Trigram LM with add-k smoothing."""
    order_n = 3

    def __eq__(self, other):
        """Do not modify."""
        state_vars = ['k', 'counts', 'context_totals', 'words', 'V']
        return all([getattr(self, v) == getattr(other, v) for v in state_vars])

    def __init__(self, tokens):
        """Build our trigram model.
        Args:
          tokens: (list or np.array) of training tokens
        Returns:
          None
        """
        print("Num tokens: ", len(tokens))
        self.k = 0.0
        # Raw trigram counts over the corpus.
        # c(w | w_1 w_2) = self.counts[(w_2,w_1)][w]
        # Be sure to use tuples (w_2,w_1) as keys, *not* lists [w_2,w_1]
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))

        # Map of (w_1, w_2) -> int
        # Entries are c( w_2, w_1 ) = sum_w c(w_2, w_1, w)
        self.context_totals = dict()

        # Track unique words seen, for normalization
        # Use wordset.add(word) to add words
        wordset = set()

        # Iterate through the word stream once
        # Compute trigram counts 
        # This is a sliding window over each word.
        w_1, w_2 = None, None
        for word in tokens:
            wordset.add(word)
            if w_1 is not None and w_2 is not None:
                self.counts[(w_2,w_1)][word] += 1
            # Update context
            w_2 = w_1
            w_1 = word

        for context, ctr in iter(self.counts.items()):  
            self.context_totals[context] = sum(iter(ctr.values())) 

        # Total vocabulary size, for normalization
        self.vocab = wordset
        self.V = len(self.vocab)

    def set_k(self, k=0.0, **params):
        self.k = k

    def next_token_conditional_prob(self, word, seq):
        """Next token conditional probability.
        Args:
          word: (string) w in P(w | w_1 w_2 )
          seq: (list of string) [w_1, w_2, w_3, ...]
        Returns:
          (float) P_k(w | w_1 w_2), according to the model
        """
        context = tuple(seq[-2:])  # (w_2, w_1)
        k = self.k
      
        cw = self.counts.get(context, {}).get(word, 0)  
        numerator = cw + k  # add k smoothing
        cc = self.context_totals.get(context, 0)  
        denominator = cc + (self.V * k)  
        return numerator / denominator  
      
    def sequence_probability(self, seq, verbose=False):
      """Compute log probability (base 2) of the given sequence."""
      context_size = self.order_n - 1
      score = 0.0
      count = 0
      # Start at third word, since we need a full context.
      for i in range(context_size, len(seq)):
        if (seq[i] == "<p>" or seq[i] == "</p>"):
            continue  # Don't count special tokens in score.
        context = seq[i-context_size:i]
        context_prob = self.next_token_conditional_prob(seq[i], context)
        s = np.log2(context_prob)
        score += s
        count += 1
        # DEBUG.
        if verbose:
            print ("log P(%s | %s) = %.03f" % (seq[i], " ".join(context), s))
      return score, count      

In [0]:
smoothed_trigram_lm = AddKTrigramLM(flat_delimited_train_tokens)


#### Exercise
- Evaluate the perplexity of the model with different values of K. 
- What is the best value for k you can find? 
- What parameter should you use? And why?

Note: Each iteration may take time -- about 10-20 seconds in colab, so be careful how many parameters you test. 

In [0]:
K = [0.000001, 0.0001, 1.0, 2.0]

for k in K:
  print("Trying k value: ", k)
  smoothed_trigram_lm.set_k(k=k)
  log_p_data, num_real_tokens = smoothed_trigram_lm.sequence_probability(flat_delimited_train_tokens)
  print ("Train perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens)))


  log_p_data, num_real_tokens = smoothed_trigram_lm.sequence_probability(flat_delimited_test_tokens)
  print("Test perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens)))
  print()

What happens to the train/test perplexities as the value increases?  Why should we expect this?
A curious thing happens with the test perplexity. It starts very high (remember it was infinite without smoothing), it then decreases indicating that the model fits the data better.  Finally, both train and test perplexity increase dramatically as we move away from good parameters.  This is typical of many ML algorithms.

## Wrap up

Let's go back to our original spelling corrector and see if we can do better with our trigram language model. Recall that the naive spelling corrector made silly mistakes:

*   it is amazon -> it is amazon
*   http www amazin -> http www amazing



If we used our trigram model, could we do better? Let's see how our model would perform by scoring these sequences.
**Note: **Change the k value to the best value you found.

In [0]:
# [it is amazon]
smoothed_trigram_lm.set_k(k=0.0001)
print(smoothed_trigram_lm.sequence_probability(["it", "is", "amazon"], verbose=False)[0])

# [it is amazing]
print(smoothed_trigram_lm.sequence_probability(["it", "is", "amazing"], verbose=False)[0])


The unsmoothed probability would be infinite; with our smoothed model the probabilities of the incorrect sequences are infinitessimal compared with the correct ones. Note that log probabilities are always negative, so the smaller magnitude is better. And remember the log scale: a difference of score of 10 units means one utterance is $2^{10} = 1K$ times more likely!

In practice, spelling correctors are more complicated than this simple model. A real spelling corrector must also consider all possible sequences of words and find the most likely sequence! We'll see how this work when we discuss NLP in coming weeks. If you'd like to read more about an application in practice, feel free to read about how shopping service Etsy uses language models in [spelling correction in search](https://codeascraft.com/2017/05/01/modeling-spelling-correction-for-search-at-etsy/).  

# End

This is the end of Lab 3. Please leave us a little feedback about the lab [on the Moodle quiz](https://moodle.gla.ac.uk/mod/feedback/view.php?id=1114113).

There is another application of language modelling in the Extra Material below if you would like more fun. Peter Norvig's book 


## Extra material

### Application 2: Text Segmentation

You can see we can apply this **really** simple model to another neat application[1]!

In the two cells below, we test out our model by taking a piece of text with all the spaces removed, enumerating all possible ways of splitting it (efficiently using dynamic programming), scoring each, and then returning the highest scoring sequence.

[1] Peter Norvig, a director of research at Google implemented this model here:  http://norvig.com/ngrams/ch14.pdf and extends it pretty far.  By the end, he has managed to decrypt WWI encryption using only this simple language model!

In [0]:
import math

class SimpleSegmenter(object): 
  
  def __init__(self, dict):
    self.dict = dict
  
  def memo(self, fn):
    cache = {}
    def docache(arg):
      if arg in cache:
        return cache[arg]
      val = fn(arg)
      cache[arg] = val
      return val
    return docache

  #@memo
  def segment(self, text):
    if not text: return []
    candidates = ([first]+self.segment(rem) for first, rem in self.splits(text))
    return max(candidates, key=lambda w: self.Pwords(w))

  def splits(self, text, L=20):
    return [(text[:i+1], text[i+1:])
            for i in range(min(len(text), L))]

  def Pw(self, w):
      # We see here that we often perform operations in log space to avoid issues
      # of floating point error with very small numbers.
      if w in self.dict.vocab_set:
          return math.log(self.dict.token_counts[w]) - math.log(self.dict.N)
      else:
          return math.log(self.dict.N) - 100*len(w)

  def Pwords(self, words):
    return sum(self.Pw(w) for w in words)

In [0]:
segmenter = SimpleSegmenter(dictionary)
print(segmenter.segment('hellotherehowareyou'))
print(segmenter.segment('tryyourstringhere'))


We could use this segmenter to optimally tokenize URLs, for example.  This segmenter just uses the simple unigram model, but it could also use more complex LMs.

**Question**: What's the smoothing used in this probability?